In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [3]:
#device Config
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Hyper parameters
input_size=784
hidden_size=100
num_classes=10
num_epochs=2
batch_size=100
learning_rate=0.001
train_dataset=torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset=torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
train_loader=torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
examples=iter(train_loader)
samples,labels=examples.next()
print(samples.shape, labels.shape)

#for i in range(6):
#    plt.subplot(2,3, i+1)
#    plt.imshow(samples[i][0], cmap='gray')
#plt.show()

torch.Size([100, 1, 28, 28]) torch.Size([100])


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1=nn.Linear(input_size, hidden_size)
        self.relu=nn.ReLU()
        self.l2=nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out=self.l1(x)
        out=self.relu(out)
        out=self.l2(out)
        return out
model=NeuralNet(input_size,hidden_size,num_classes)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)
#training_loop
n_total_steps=len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images=images.reshape(-1,28*28).to(device)
        labels=labels.to(device)
        #forward
        outputs=model(images)
        loss=criterion(outputs,labels)
        #backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #print the loss
        if (i+1)%100==0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss={loss.item():.4f}')
# test
with torch.no_grad():
    n_correct=0
    n_samples=0
    for images,labels in test_loader:
        images=images.reshape(-1, 28*28).to(device)
        labels=labels.to(device)
        outputs=model(images)
        #value, index
        _,predictions=torch.max(outputs,1)
        n_samples=labels.shape[0]
        n_correct=(predictions==labels).sum().item()
acc=100.0*n_samples/n_samples
print(acc)

epoch 1 / 2, step 100/600, loss=0.3815
epoch 1 / 2, step 200/600, loss=0.3858
epoch 1 / 2, step 300/600, loss=0.2204
epoch 1 / 2, step 400/600, loss=0.2856
epoch 1 / 2, step 500/600, loss=0.5379
epoch 1 / 2, step 600/600, loss=0.3394
